In [2]:
#!pip install python-Levenshtein

In [3]:
# import some libraries
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz

C:\Users\jenny_lai\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
# read in data
# data sets contains:
# swimming pool
# martial art
# park
# athlete

swim = pd.read_csv('swimming club location information v1_0.csv')
martial = pd.read_csv('martial art location information v1_0.csv')
park = pd.read_csv('park and reserve location information v1_0.csv')
athlete = pd.read_csv('athlete club location information v1_0.csv')



In [5]:
# concate these tables
sports = pd.concat([swim, martial, park, athlete], axis=0, ignore_index=True)
sports.head(10)

,Unnamed: 0,orgName,orgAddress,searchquery,Name,formattedAddress,State,Business Category,longitude,latitude,type,status_code,next_page_token,price_level,rating,opennow,permanent_closed,place_id
0,0,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",VIC,Swimming Club,145.041892,-37.724723,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.0,False,NaN,ChIJz6KE4uZF1moRYpNq83r9-Bc
1,1,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",VIC,Swimming Club,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",OK,NaN,NaN,4.3,NaN,NaN,ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,2,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",VIC,Swimming Club,145.041096,-37.723869,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.2,NaN,NaN,ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,3,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",VIC,Swimming Club,145.042285,-37.723135,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.2,NaN,NaN,ChIJzzKEpOZF1moR1tL2Y-oSbA8
4,4,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia",VIC,Swimming Club,145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",OK,NaN,NaN,3.9,False,NaN,ChIJHXzMjuJF1moRbamYB4aaEas
5,5,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Cricket Club,"Bundoora VIC 3083, Australia",VIC,Swimming Club,145.042839,-37.722393,"['point_of_interest', 'establishment']",OK,NaN,NaN,5.0,NaN,NaN,ChIJgejvG-RF1moR0eUlf49nngI
6,6,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe College Australia,"Level 3, Sylvia Walton Building, Bundoora VIC ...",VIC,Swimming Club,145.050119,-37.721764,"['university', 'library', 'point_of_interest',...",OK,NaN,NaN,4.2,False,NaN,ChIJ6QUDCstC1moRtcFHQWj5wDs
7,7,Bairnsdale Amateur Swimming Club,NaN,Bairnsdale+Amateur+Swimming+Club++Bairnsdale,Bairnsdale Aquatic & Recreation Centre,"80 McKean St, Bairnsdale VIC 3875, Australia",VIC,Swimming Club,147.605199,-37.826882,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.1,False,NaN,ChIJB0B5zAR_L2sRqVfxo7ODHsY
8,8,Bairnsdale Amateur Swimming Club,NaN,Bairnsdale+Amateur+Swimming+Club++Bairnsdale,Bairnsdale Outdoor Pool,"49 Ligar St, Bairnsdale VIC 3875, Australia",VIC,Swimming Club,147.618296,-37.823043,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.0,False,NaN,ChIJb3hVYQp_L2sR0x3UJzzIcTk
9,9,Ballarat GCO Swim Club,NaN,Ballarat+GCO+Swim+Club++Ballarat,Ballarat GCO Swim Club,NaN,VIC,NaN,NaN,NaN,NaN,ZERO_RESULTS,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
sports.shape

(951, 18)

In [7]:
sports.dtypes

Unnamed: 0             int64
orgName               object
orgAddress            object
searchquery           object
Name                  object
formattedAddress      object
State                 object
Business Category     object
longitude            float64
latitude             float64
type                  object
status_code           object
next_page_token       object
price_level          float64
rating               float64
opennow               object
permanent_closed      object
place_id              object
dtype: object

In [8]:
# check columns
sports.columns

Index(['Unnamed: 0', 'orgName', 'orgAddress', 'searchquery', 'Name',
       'formattedAddress', 'State', 'Business Category', 'longitude',
       'latitude', 'type', 'status_code', 'next_page_token', 'price_level',
       'rating', 'opennow', 'permanent_closed', 'place_id'],
      dtype='object')

In [9]:
# delete the first column
sports.drop(columns='Unnamed: 0',inplace=True)
sports.head(10)

,orgName,orgAddress,searchquery,Name,formattedAddress,State,Business Category,longitude,latitude,type,status_code,next_page_token,price_level,rating,opennow,permanent_closed,place_id
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",VIC,Swimming Club,145.041892,-37.724723,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.0,False,NaN,ChIJz6KE4uZF1moRYpNq83r9-Bc
1,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",VIC,Swimming Club,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",OK,NaN,NaN,4.3,NaN,NaN,ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",VIC,Swimming Club,145.041096,-37.723869,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.2,NaN,NaN,ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",VIC,Swimming Club,145.042285,-37.723135,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.2,NaN,NaN,ChIJzzKEpOZF1moR1tL2Y-oSbA8
4,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia",VIC,Swimming Club,145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",OK,NaN,NaN,3.9,False,NaN,ChIJHXzMjuJF1moRbamYB4aaEas
5,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Cricket Club,"Bundoora VIC 3083, Australia",VIC,Swimming Club,145.042839,-37.722393,"['point_of_interest', 'establishment']",OK,NaN,NaN,5.0,NaN,NaN,ChIJgejvG-RF1moR0eUlf49nngI
6,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe College Australia,"Level 3, Sylvia Walton Building, Bundoora VIC ...",VIC,Swimming Club,145.050119,-37.721764,"['university', 'library', 'point_of_interest',...",OK,NaN,NaN,4.2,False,NaN,ChIJ6QUDCstC1moRtcFHQWj5wDs
7,Bairnsdale Amateur Swimming Club,NaN,Bairnsdale+Amateur+Swimming+Club++Bairnsdale,Bairnsdale Aquatic & Recreation Centre,"80 McKean St, Bairnsdale VIC 3875, Australia",VIC,Swimming Club,147.605199,-37.826882,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.1,False,NaN,ChIJB0B5zAR_L2sRqVfxo7ODHsY
8,Bairnsdale Amateur Swimming Club,NaN,Bairnsdale+Amateur+Swimming+Club++Bairnsdale,Bairnsdale Outdoor Pool,"49 Ligar St, Bairnsdale VIC 3875, Australia",VIC,Swimming Club,147.618296,-37.823043,"['point_of_interest', 'establishment']",OK,NaN,NaN,4.0,False,NaN,ChIJb3hVYQp_L2sR0x3UJzzIcTk
9,Ballarat GCO Swim Club,NaN,Ballarat+GCO+Swim+Club++Ballarat,Ballarat GCO Swim Club,NaN,VIC,NaN,NaN,NaN,NaN,ZERO_RESULTS,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# in current iteration: price_level and rating are not necessary
# delete
sports.drop(columns=['price_level','rating'],inplace=True)
sports.head()

,orgName,orgAddress,searchquery,Name,formattedAddress,State,Business Category,longitude,latitude,type,status_code,next_page_token,opennow,permanent_closed,place_id
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",VIC,Swimming Club,145.041892,-37.724723,"['point_of_interest', 'establishment']",OK,NaN,False,NaN,ChIJz6KE4uZF1moRYpNq83r9-Bc
1,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",VIC,Swimming Club,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",OK,NaN,NaN,NaN,ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",VIC,Swimming Club,145.041096,-37.723869,"['point_of_interest', 'establishment']",OK,NaN,NaN,NaN,ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",VIC,Swimming Club,145.042285,-37.723135,"['point_of_interest', 'establishment']",OK,NaN,NaN,NaN,ChIJzzKEpOZF1moR1tL2Y-oSbA8
4,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia",VIC,Swimming Club,145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",OK,NaN,False,NaN,ChIJHXzMjuJF1moRbamYB4aaEas


In [11]:
# delete next page token
sports.drop(columns='next_page_token',inplace=True)

In [12]:
# check status code
print(sports.status_code.unique())
# inspect not valid resuts or zero result
sports[sports.status_code != 'OK'].head()

['OK' 'ZERO_RESULTS' 'INVALID_REQUEST']


,orgName,orgAddress,searchquery,Name,formattedAddress,State,Business Category,longitude,latitude,type,status_code,opennow,permanent_closed,place_id
9,Ballarat GCO Swim Club,NaN,Ballarat+GCO+Swim+Club++Ballarat,Ballarat GCO Swim Club,NaN,VIC,NaN,NaN,NaN,NaN,ZERO_RESULTS,NaN,NaN,NaN
24,Heidelberg Flyers AUSSI Masters,Heidelberg Aquatic Centre,Heidelberg+Flyers+AUSSI+Masters+Heidelberg+Aqu...,Heidelberg Flyers AUSSI Masters,NaN,VIC,NaN,NaN,NaN,NaN,ZERO_RESULTS,NaN,NaN,NaN
26,Ivanhoe Aqua Power Swimming Club,123 Upper Heidelberg Street,Ivanhoe+Aqua+Power+Swimming+Club+123+Upper+Hei...,Ivanhoe Aqua Power Swimming Club,NaN,VIC,NaN,NaN,NaN,NaN,ZERO_RESULTS,NaN,NaN,NaN
29,Kilmore Country Masters Swimming Club,White Street,Kilmore+Country+Masters+Swimming+Club+White+St...,Kilmore Country Masters Swimming Club,NaN,VIC,NaN,NaN,NaN,NaN,ZERO_RESULTS,NaN,NaN,NaN
30,Kilmore District Amateur Swimming Club,White Street,Kilmore+District+Amateur+Swimming+Club+White+S...,Kilmore District Amateur Swimming Club,NaN,VIC,NaN,NaN,NaN,NaN,ZERO_RESULTS,NaN,NaN,NaN


In [13]:
# keep only records with status code: OK
# store in sports1
sports1 = sports[sports.status_code == 'OK']

In [14]:
# delete status_code after filtering
sports1.drop(columns='status_code',inplace=True)
sports1.head()

C:\Users\jenny_lai\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,orgName,orgAddress,searchquery,Name,formattedAddress,State,Business Category,longitude,latitude,type,opennow,permanent_closed,place_id
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",VIC,Swimming Club,145.041892,-37.724723,"['point_of_interest', 'establishment']",False,NaN,ChIJz6KE4uZF1moRYpNq83r9-Bc
1,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",VIC,Swimming Club,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",NaN,NaN,ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",VIC,Swimming Club,145.041096,-37.723869,"['point_of_interest', 'establishment']",NaN,NaN,ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",VIC,Swimming Club,145.042285,-37.723135,"['point_of_interest', 'establishment']",NaN,NaN,ChIJzzKEpOZF1moR1tL2Y-oSbA8
4,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia",VIC,Swimming Club,145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",False,NaN,ChIJHXzMjuJF1moRbamYB4aaEas


In [15]:
# check whether there are facility permanently closed
sports1.permanent_closed.unique()

array([nan, True], dtype=object)

In [16]:
# delete those permanently closed
sports1 = sports1[sports1.permanent_closed != True]

In [17]:
# drop column: permanent_closed
sports1.drop(columns='permanent_closed',inplace=True)
sports1.head()

,orgName,orgAddress,searchquery,Name,formattedAddress,State,Business Category,longitude,latitude,type,opennow,place_id
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",VIC,Swimming Club,145.041892,-37.724723,"['point_of_interest', 'establishment']",False,ChIJz6KE4uZF1moRYpNq83r9-Bc
1,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",VIC,Swimming Club,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",NaN,ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",VIC,Swimming Club,145.041096,-37.723869,"['point_of_interest', 'establishment']",NaN,ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",VIC,Swimming Club,145.042285,-37.723135,"['point_of_interest', 'establishment']",NaN,ChIJzzKEpOZF1moR1tL2Y-oSbA8
4,Aquabear AUSSI Masters,La Trobe University Sports Centre,Aquabear+AUSSI+Masters+La+Trobe+University+Spo...,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia",VIC,Swimming Club,145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",False,ChIJHXzMjuJF1moRbamYB4aaEas


In [18]:
# drop column: searchquery
sports1.drop(columns='searchquery',inplace=True)

In [19]:
# in current iteration: opennow is not necessary
# drop column: opennow
sports1.drop(columns='opennow',inplace=True)
sports1.head()

,orgName,orgAddress,Name,formattedAddress,State,Business Category,longitude,latitude,type,place_id
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",VIC,Swimming Club,145.041892,-37.724723,"['point_of_interest', 'establishment']",ChIJz6KE4uZF1moRYpNq83r9-Bc
1,Aquabear AUSSI Masters,La Trobe University Sports Centre,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",VIC,Swimming Club,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",VIC,Swimming Club,145.041096,-37.723869,"['point_of_interest', 'establishment']",ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",VIC,Swimming Club,145.042285,-37.723135,"['point_of_interest', 'establishment']",ChIJzzKEpOZF1moR1tL2Y-oSbA8
4,Aquabear AUSSI Masters,La Trobe University Sports Centre,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia",VIC,Swimming Club,145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",ChIJHXzMjuJF1moRbamYB4aaEas


In [20]:
# delete state
sports1.drop(columns='State',inplace=True)

In [21]:
sports1.columns

Index(['orgName', 'orgAddress', 'Name', 'formattedAddress',
       'Business Category', 'longitude', 'latitude', 'type', 'place_id'],
      dtype='object')

In [22]:
sports1.shape

(850, 9)

In [23]:
b_uni = sports1['Business Category'].unique()
b_uni

array(['Swimming Club', 'Swimming Club,Swimming Pool', 'Swimming Pool',
       'Swimming Pool,Water Based Exercise',
       'Disability,Fitness centre,Strength Training,Swimming Pool',
       'Disability,Fitness centre,Strength Training,Swimming Pool,Water Based Exercise',
       'Disability,Swimming Club,Swimming Pool',
       'Disability,Swimming Club,Wheelchair Sports',
       'Disability,Swimming Pool',
       'Disability,Strength Training,Swimming Pool,Water Based Exercise',
       'Disability,Recreation Group,Swimming Pool',
       'Fitness centre,Gymnastics,Personal training,Strength Training,Swimming Pool',
       'Fitness centre,Personal training,Strength Training,Swimming Club,Swimming Pool,Water Based Exercise',
       'Fitness centre,Personal training,Swimming Pool',
       'Fitness centre,Squash Club,Swimming Club,Tenpin Bowling',
       'Fitness centre,Strength Training,Swimming Pool',
       'Fitness centre,Strength Training,Swimming Pool,Water Based Exercise',
       'F

In [24]:
# check unique business type
b_list = []
for i in range(len(b_uni)):
    b_uni_sub = b_uni[i].split(',')
    for j in range(len(b_uni_sub)):
        b_list.append(b_uni_sub[j])
    
len(b_list)

82

In [25]:
# check unique business type
# use set()
b_list = set(b_list)
b_list

{'Athletics Club',
 'Basketball Club',
 'Disability',
 'Fitness centre',
 'Gymnastics',
 'Little Athletics',
 'Martial Arts',
 'Park/Garden/Reserve',
 'Personal training',
 'Recreation Group',
 'Softball Club',
 'Sports Club/Association',
 'Squash Club',
 'State body',
 'Strength Training',
 'Swimming Club',
 'Swimming Pool',
 'Tai Chi',
 'Tenpin Bowling',
 'Water Based Exercise',
 'Wheelchair Sports'}

In [26]:
# new data frame
sports_check = pd.DataFrame(columns=['orgName', 'orgAddress', 'Name', 'formattedAddress',
                                     'Business Category', 'newB',
                                     'longitude', 'latitude', 'type',
                                     'place_id'])




In [27]:
# append detail in new data frame
new_record = {'orgName':None,
              'orgAddress':None,
              'Name':None,
              'formattedAddress':None,
              'longitude':None,
              'latitude':None,
              'type': None,
              'place_id':None,
              'Business Category':[],
              'newB':None}
new_record

{'Business Category': [],
 'Name': None,
 'formattedAddress': None,
 'latitude': None,
 'longitude': None,
 'newB': None,
 'orgAddress': None,
 'orgName': None,
 'place_id': None,
 'type': None}

In [28]:
# fuzzy match
s1 = 'Waverley Road'
s2 = 'waverley'

fuzz.ratio(s1.lower(), s2.lower())

76

In [29]:
# sports_t: sports test for fuzzy
# set copy of sports1
sports_t = sports1.copy()
sports_t.head()

,orgName,orgAddress,Name,formattedAddress,Business Category,longitude,latitude,type,place_id
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",Swimming Club,145.041892,-37.724723,"['point_of_interest', 'establishment']",ChIJz6KE4uZF1moRYpNq83r9-Bc
1,Aquabear AUSSI Masters,La Trobe University Sports Centre,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",Swimming Club,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",Swimming Club,145.041096,-37.723869,"['point_of_interest', 'establishment']",ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",Swimming Club,145.042285,-37.723135,"['point_of_interest', 'establishment']",ChIJzzKEpOZF1moR1tL2Y-oSbA8
4,Aquabear AUSSI Masters,La Trobe University Sports Centre,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia",Swimming Club,145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",ChIJHXzMjuJF1moRbamYB4aaEas


In [30]:
sports_t.shape

(850, 9)

In [31]:
# new data frame
sports_c = pd.DataFrame(columns=['Name', 'formattedAddress',
                                 'newB',
                                 'longitude', 'latitude', 'type',
                                 'place_id'])


# append detail in new data frame
# for sports_c
new_record = {'Name':None,
              'formattedAddress':None,
              'longitude':None,
              'latitude':None,
              'type': None,
              'place_id':None,
              'Business Category':[],
              'newB':None}
new_record

{'Business Category': [],
 'Name': None,
 'formattedAddress': None,
 'latitude': None,
 'longitude': None,
 'newB': None,
 'place_id': None,
 'type': None}

In [32]:
# check park and reserve
# assumptions here, if the two club or place names highly match,
# identify as the similar place
# share the same business type
# test for threshold
# threshold: 75
orgname = ''
name = ''
count = 0

sports_c = pd.DataFrame(columns=['Name', 'formattedAddress',
                                 'newB',
                                 'longitude', 'latitude', 'type',
                                 'place_id'])


for i in range(sports_t.shape[0]):
    orgname = sports_t.iloc[i,:].orgName
    name = sports_t.iloc[i,:].Name
    
    # new record
    new_record = {'Name':None,
                  'formattedAddress':None,
                  'longitude':None,
                  'latitude':None,
                  'type': None,
                  'place_id':None,
                  #'Business Category':[],
                  'newB':None}
    if fuzz.ratio(orgname.lower(), name.lower()) >= 75:
        count += 1
        #print(sports_t.iloc[i,:])
        #print(orgname, name)
        new_record = {'Name':name,
                      'formattedAddress':sports_t.iloc[i,:].formattedAddress,
                      'longitude':sports_t.iloc[i,:].longitude,
                      'latitude':sports_t.iloc[i,:].latitude,
                      'type': sports_t.iloc[i,:].type,
                      'place_id':sports_t.iloc[i,:].place_id,
                      #'Business Category':sports_t.iloc[i,:]['Business Category'],
                      'newB':sports_t.iloc[i,:]['Business Category']}
        
        # append new record
        sports_c = sports_c.append(new_record, ignore_index=True)
        #sports_t = sports_t.iloc[-5,:]
        
print('done')

done


In [33]:
count

362

In [34]:
# check data frame
sports_c.head()

,Name,formattedAddress,newB,longitude,latitude,type,place_id
0,Eltham Swimming School,"1441 Main Rd, Eltham VIC 3095, Australia",Swimming Club,145.175774,-37.701532,"['point_of_interest', 'establishment']",ChIJnc-kCFw21moR-O84vUwjjTM
1,Jayson Lamb’s Splash Factory,"Grafton Rd, Warrnambool VIC 3280, Australia",Swimming Club,142.501409,-38.379052,"['point_of_interest', 'establishment']",ChIJV5zTXLpNzWoRKdMXeO49xpE
2,Just Swimming Wesley,"Wesley College, 554 High St Rd, Mount Waverley...",Swimming Club,145.146906,-37.872370,"['health', 'point_of_interest', 'school', 'est...",ChIJZ28toY0_1moRIKoSaJU36AM
3,Melbourne Vicentre Swimming Club,"Aughtie Dr, Albert Park VIC 3206, Australia",Swimming Club,144.960939,-37.841647,"['point_of_interest', 'establishment']",ChIJud2iLApo1moRpnfMpGO_mcQ
4,Melbourne Vicentre Swimming Club,"Ranfurlie Cres, Glen Iris VIC 3146, Australia",Swimming Club,145.054289,-37.861586,"['health', 'point_of_interest', 'school', 'est...",ChIJk4SJNWBA1moRxEt3tTgM20A


In [35]:
# check duplicated
sports_c[sports_c.duplicated(subset=['place_id'],keep=False)]

,Name,formattedAddress,newB,longitude,latitude,type,place_id
141,Dandenong Ranges National Park,"Sassafras VIC 3787, Australia",Park/Garden/Reserve,145.347640,-37.860914,"['park', 'point_of_interest', 'establishment']",ChIJqeipof4k1moREPExBXZWBA8
168,Dandenong Ranges National Park,"Sassafras VIC 3787, Australia",Park/Garden/Reserve,145.347640,-37.860914,"['park', 'point_of_interest', 'establishment']",ChIJqeipof4k1moREPExBXZWBA8
301,Wilsons Promontory National Park,"Wilsons Promontory VIC 3960, Australia",Park/Garden/Reserve,146.274980,-39.109205,"['park', 'point_of_interest', 'establishment']",ChIJu9J6SB6pK2sR5Napl7hrmlI
302,Wilsons Promontory National Park,"Wilsons Promontory VIC 3960, Australia",Park/Garden/Reserve,146.274980,-39.109205,"['park', 'point_of_interest', 'establishment']",ChIJu9J6SB6pK2sR5Napl7hrmlI
303,Wilsons Promontory National Park,"Wilsons Promontory VIC 3960, Australia",Park/Garden/Reserve,146.274980,-39.109205,"['park', 'point_of_interest', 'establishment']",ChIJu9J6SB6pK2sR5Napl7hrmlI
317,Diamond Valley Little Athletics Centre,"21A Talbot St, Greensborough VIC 3088, Australia",Athletics Club,145.109426,-37.710961,"['point_of_interest', 'establishment']",ChIJ_X3ThUJI1moRaNmWgDrRJnk
334,Sandringham Athletics Club,"Glamis Ave, Hampton VIC 3188, Australia","Athletics Club,Sports Club/Association",145.016758,-37.939216,"['gym', 'health', 'point_of_interest', 'establ...",ChIJSTqr1Nxu1moRSDal3vHTd5U
342,Diamond Valley Little Athletics Centre,"21A Talbot St, Greensborough VIC 3088, Australia",Little Athletics,145.109426,-37.710961,"['point_of_interest', 'establishment']",ChIJ_X3ThUJI1moRaNmWgDrRJnk
351,Sandringham Athletics Club,"Glamis Ave, Hampton VIC 3188, Australia",Little Athletics,145.016758,-37.939216,"['gym', 'health', 'point_of_interest', 'establ...",ChIJSTqr1Nxu1moRSDal3vHTd5U


In [36]:
p_list = sports_c.place_id.unique()
len(p_list)

357

In [37]:
# after check similarity
# new data frame
# filter out those in sports_c
sports_t2 = pd.DataFrame(columns=['orgName', 'orgAddress', 'Name', 'formattedAddress',
                                     'Business Category', 'newB',
                                     'longitude', 'latitude', 'type',
                                     'place_id'])

# append detail in new data frame
new_t2 = {'orgName':None,
              'orgAddress':None,
              'Name':None,
              'formattedAddress':None,
              'longitude':None,
              'latitude':None,
              'type': None,
              'place_id':None,
              'Business Category':None,
              'newB':None}
new_t2


for i in range(sports_t.shape[0]):
    new_t2 = {'orgName':None,
                  'orgAddress':None,
                  'Name':None,
                  'formattedAddress':None,
                  'longitude':None,
                  'latitude':None,
                  'type': None,
                  'place_id':None,
                  'Business Category':None,
                  'newB':None}
    if sports_t.iloc[i,:].place_id not in p_list:
        new_t2 = {'orgName':sports_t.iloc[i,:].orgName,
                  'orgAddress':sports_t.iloc[i,:].orgAddress,
                  'Name':sports_t.iloc[i,:].Name,
                  'formattedAddress':sports_t.iloc[i,:].formattedAddress,
                  'longitude':sports_t.iloc[i,:].longitude,
                  'latitude':sports_t.iloc[i,:].latitude,
                  'type': sports_t.iloc[i,:].type,
                  'place_id':sports_t.iloc[i,:].place_id,
                  'Business Category':sports_t.iloc[i,:]['Business Category'],
                  'newB':None}
        # keep those not in sports_c as sports_t2
        sports_t2 = sports_t2.append(new_t2,ignore_index=True)

In [38]:
sports_t2.head()

,orgName,orgAddress,Name,formattedAddress,Business Category,newB,longitude,latitude,type,place_id
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",Swimming Club,None,145.041892,-37.724723,"['point_of_interest', 'establishment']",ChIJz6KE4uZF1moRYpNq83r9-Bc
1,Aquabear AUSSI Masters,La Trobe University Sports Centre,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",Swimming Club,None,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",Swimming Club,None,145.041096,-37.723869,"['point_of_interest', 'establishment']",ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",Swimming Club,None,145.042285,-37.723135,"['point_of_interest', 'establishment']",ChIJzzKEpOZF1moR1tL2Y-oSbA8
4,Aquabear AUSSI Masters,La Trobe University Sports Centre,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia",Swimming Club,None,145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",ChIJHXzMjuJF1moRbamYB4aaEas


In [39]:
sports_t2.shape

(437, 10)

In [40]:
# new data frame
# deal with sports_t2
sports_c2 = pd.DataFrame(columns=['Name', 'formattedAddress',
                                  'newB',
                                 'longitude', 'latitude', 'type',
                                 'place_id'])


# append detail in new data frame
# for sports_c2
new_record2 = {'Name':None,
              'formattedAddress':None,
              'longitude':None,
              'latitude':None,
              'type': None,
              'place_id':None,
              #'Business Category':None,
              'newB':None}
new_record2

{'Name': None,
 'formattedAddress': None,
 'latitude': None,
 'longitude': None,
 'newB': None,
 'place_id': None,
 'type': None}

In [41]:
# deal with sports_t2
name = ''


sports_c2 = pd.DataFrame(columns=['Name', 'formattedAddress',
                                  'newB',
                                  'longitude', 'latitude', 'type',
                                  'place_id'])


for i in range(sports_t2.shape[0]):
    new_record2 = {'Name':None,
                  'formattedAddress':None,
                  'longitude':None,
                  'latitude':None,
                  'type': None,
                  'place_id':None,
                  #'Business Category':None,
                  'newB':None}
    
    # for swimming facility
    name = sports_t2.iloc[i,:].Name.lower()
    t = sports_t2.iloc[i,:].type 
    # newset to store business category
    newset = set()
    #print(type(name))
    
    # parsing category
    # parse golf or chess or tai chi
    # type we don't need
    if 'chess' in name or 'golf' in name or 'tai chi' in name:
        #print(name)
        continue
    
    # parse swimming facility
    elif 'swim' in name or 'swimming' in name or 'aquatic' in name or 'pool' in name:
        #print(name)
        newset.add('Swimming Pool')
        new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                      'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                      'longitude':sports_t2.iloc[i,:].longitude,
                      'latitude':sports_t2.iloc[i,:].latitude,
                      'type': sports_t2.iloc[i,:].type,
                      'place_id':sports_t2.iloc[i,:].place_id,
                      #'Business Category':sports_check.iloc[i,:]['Business Category'],
                      'newB':newset
                      }
        # for swimming club
        if 'learn' in name or 'school' in name or 'club' in name or 'academy' in name:
            newset.add('Swimming Club')
            new_record2['newB'] = newset

    elif 'fitness' in name:
        # for an old record
        #print(name)
        if new_record2.get('place_id') is not None:
            newset.add('Fitness Centre')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Fitness Centre')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
        
    elif 'recreation' in name:
        # for an old record
        #print(name)
        if new_record2.get('place_id') is not None:
            newset.add('Fitness Centre')
            newset.add('Recreation Centre')
            newset.add('Swimming Pool')
            newset.add('Gymnastics')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Fitness Centre')
            newset.add('Recreation Centre')
            newset.add('Swimming Pool')
            newset.add('Gymnastics')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
        
        
    # athletic club    
    elif 'athlete' in name or 'athletic' in name or 'athletics' in name:
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Athletics Club')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Athletics Club')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
            
        
        if 'little athletics' in name:
            newset.add('Little Athletics')
            new_record2['newB'] = newset
            
    elif 'stadium' in t:
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Strength Training')
            newset.add('Fitness Centre')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Strength Training')
            newset.add('Fitness Centre')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
    
    # martial art club
    elif 'martial arts' in name or\
    'judo' in name or \
    'aikido' in name or \
    'taekwondo' in name or\
    'karate' in name or \
    'kickboxing' in name or\
    'kung fu' in name:
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Strength Training')
            newset.add('Martial Arts')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Strength Training')
            newset.add('Martial Arts')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
    
    # park and reserve
    elif 'park' in name or 'reserve' in name or 'camp' in name or 'area' in name or 'site' in name or 'settlement' in name:
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Park/Garden/Reserve')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Park/Garden/Reserve')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
            
            
    elif 'park' in t or 'natural_feature' in t:
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Park/Garden/Reserve')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Park/Garden/Reserve')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
            
            
    elif 'tennis' in name :
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Tennis Club')
            #newset.add('Sports Club/Association')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Tennis Club')
            #newset.add('Sports Club/Association')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
            
        if 'association' in name:
            newset.add('Sports Club/Association')
            new_record2['newB'] = newset
            
    elif 'basketball' in name :
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Basketball Club')
            newset.add()
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Tennis Club')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
        
        if 'association' in name:
            newset.add('Sports Club/Association')
            new_record2['newB'] = newset
            
    elif 'netball' in name :
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Netball Club')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Netball Club')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
        
        if 'association' in name:
            newset.add('Sports Club/Association')
            new_record2['newB'] = newset
            

            
    elif 'soccer' in name :
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('soccer Club')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('soccer Club')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
        
        if 'association' in name:
            newset.add('Sports Club/Association')
            new_record2['newB'] = newset
            
    # baseball        
    elif 'baseball' in name :
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Baseball Club')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Baseball Club')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
        
        if 'association' in name:
            newset.add('Sports Club/Association')
            new_record2['newB'] = newset
     
    # sports centre and fitness centre
    elif 'sport' in name or 'sports' in name:
        #print(name)
        # for an old record
        if new_record2.get('place_id') is not None:
            newset.add('Strength Training')
            newset.add('Fitness centre')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Strength Training')
            newset.add('Fitness centre')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
        
        if 'association' in name:
            newset.add('Sports Club/Association')
            new_record2['newB'] = newset
            
    elif 'gym' in t or 'health' in t:
        if new_record2.get('place_id') is not None:
            newset.add('Strength Training')
            newset.add('Gymnastics')
            new_record2['newB'] = newset
        # for a new record
        else:
            newset.add('Strength Training')
            newset.add('Gymnastics')
            new_record2 = {'Name':sports_t2.iloc[i,:].Name,
                          'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                          'longitude':sports_t2.iloc[i,:].longitude,
                          'latitude':sports_t2.iloc[i,:].latitude,
                          'type': sports_t2.iloc[i,:].type,
                          'place_id':sports_t2.iloc[i,:].place_id,
                          #'Business Category':sports_check.iloc[i,:]['Business Category'],
                          'newB':newset
                          }
            
    # convert set to string and store the new type values
    newset = list(newset)
    temp = ','.join(e for e in newset)
    new_record2['newB'] = temp
    
    # append new record after parsing
    if new_record2.get('place_id') is not None:
        sports_c2 = sports_c2.append(new_record2, ignore_index=True)

print('done')

done


In [42]:
# check if the filter works
sports_c2[sports_c2.Name =='mildura golf resort']

,Name,formattedAddress,newB,longitude,latitude,type,place_id


In [43]:
sports_c2.shape

(302, 7)

In [44]:
sports_c2.head(10)

,Name,formattedAddress,newB,longitude,latitude,type,place_id
0,La Trobe University Soccer Club,"Bundoora VIC 3083, Australia",soccer Club,145.041892,-37.724723,"['point_of_interest', 'establishment']",ChIJz6KE4uZF1moRYpNq83r9-Bc
1,"La Trobe University, Melbourne Campus","Plenty Rd & Kingsbury Dr, Bundoora VIC 3086, A...",Park/Garden/Reserve,145.048414,-37.720667,"['university', 'point_of_interest', 'establish...",ChIJm6Y4iuJF1moRwNoyBXZWBA8
2,La Trobe University Baseball Club,"Bundoora campus, Kingsbury Dr, Bundoora VIC 30...",Baseball Club,145.041096,-37.723869,"['point_of_interest', 'establishment']",ChIJEXof8-ZF1moRNZzu8eHrtMQ
3,Indoor Sports Centre,"Centreway, Bundoora VIC 3086, Australia","Fitness centre,Strength Training",145.053005,-37.719812,"['gym', 'health', 'point_of_interest', 'school...",ChIJHXzMjuJF1moRbamYB4aaEas
4,Bairnsdale Aquatic & Recreation Centre,"80 McKean St, Bairnsdale VIC 3875, Australia",Swimming Pool,147.605199,-37.826882,"['point_of_interest', 'establishment']",ChIJB0B5zAR_L2sRqVfxo7ODHsY
5,Bairnsdale Outdoor Pool,"49 Ligar St, Bairnsdale VIC 3875, Australia",Swimming Pool,147.618296,-37.823043,"['point_of_interest', 'establishment']",ChIJb3hVYQp_L2sR0x3UJzzIcTk
6,Glenroy Tennis Club,"1 Ash Ct, Glenroy VIC 3046, Australia",Tennis Club,144.923319,-37.697883,"['point_of_interest', 'establishment']",ChIJ_3YyJfpa1moRmOj11BR2SGA
7,"Corpus Christi Tennis Club, Glenroy","Melbourne Ave & Cromwell St, Glenroy VIC 3046,...",Tennis Club,144.922763,-37.702880,"['point_of_interest', 'establishment']",ChIJ1dlEKvxa1moRaP6Gb0eqrRM
8,North Street Learn to Swim,"89 North St, Hadfield VIC 3046, Australia","Swimming Pool,Swimming Club",144.944056,-37.705250,"['health', 'point_of_interest', 'school', 'est...",ChIJq2Qlhdxa1moRJwzFGvQGrPI
9,Preston Reservoir Swimming Club,LaTrobe University Pool La Trobe Indoor Sports...,"Swimming Pool,Swimming Club",145.052884,-37.720054,"['point_of_interest', 'establishment']",ChIJn9xBRgZG1moRbrb35t1p5ic


In [45]:
b_list

{'Athletics Club',
 'Basketball Club',
 'Disability',
 'Fitness centre',
 'Gymnastics',
 'Little Athletics',
 'Martial Arts',
 'Park/Garden/Reserve',
 'Personal training',
 'Recreation Group',
 'Softball Club',
 'Sports Club/Association',
 'Squash Club',
 'State body',
 'Strength Training',
 'Swimming Club',
 'Swimming Pool',
 'Tai Chi',
 'Tenpin Bowling',
 'Water Based Exercise',
 'Wheelchair Sports'}

In [46]:
# check the rest of sports_c2
# as sports_t3
p2_list = sports_c2.place_id.unique()
len(p2_list)

288

In [47]:
# after check similarity
# new data frame
# filter out those in sports_c2
# sports_t3
sports_t3 = pd.DataFrame(columns=['orgName', 'orgAddress', 'Name', 'formattedAddress',
                                     'Business Category', 'newB',
                                     'longitude', 'latitude', 'type',
                                     'place_id'])

# append detail in new data frame
new_t3 = {'orgName':None,
              'orgAddress':None,
              'Name':None,
              'formattedAddress':None,
              'longitude':None,
              'latitude':None,
              'type': None,
              'place_id':None,
              'Business Category':None,
              'newB':None}
new_t3


for i in range(sports_t2.shape[0]):
    new_t3 = {'orgName':None,
              'orgAddress':None,
              'Name':None,
              'formattedAddress':None,
              'longitude':None,
              'latitude':None,
              'type': None,
              'place_id':None,
              'Business Category':None,
              'newB':None}
    if sports_t2.iloc[i,:].place_id not in p2_list:
        new_t3 = {'orgName':sports_t2.iloc[i,:].orgName,
                  'orgAddress':sports_t2.iloc[i,:].orgAddress,
                  'Name':sports_t2.iloc[i,:].Name,
                  'formattedAddress':sports_t2.iloc[i,:].formattedAddress,
                  'longitude':sports_t2.iloc[i,:].longitude,
                  'latitude':sports_t2.iloc[i,:].latitude,
                  'type': sports_t2.iloc[i,:].type,
                  'place_id':sports_t2.iloc[i,:].place_id,
                  'Business Category':sports_t2.iloc[i,:]['Business Category'],
                  'newB':None}
        # keep those not in sports_c as sports_t2
        sports_t3 = sports_t3.append(new_t3,ignore_index=True)

In [48]:
sports_t3.shape

(135, 10)

In [49]:
sports_t3.head()

,orgName,orgAddress,Name,formattedAddress,Business Category,newB,longitude,latitude,type,place_id
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Football Club,"Kingsbury Dr, Bundoora VIC 3083, Australia",Swimming Club,None,145.042285,-37.723135,"['point_of_interest', 'establishment']",ChIJzzKEpOZF1moR1tL2Y-oSbA8
1,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe University Cricket Club,"Bundoora VIC 3083, Australia",Swimming Club,None,145.042839,-37.722393,"['point_of_interest', 'establishment']",ChIJgejvG-RF1moR0eUlf49nngI
2,Aquabear AUSSI Masters,La Trobe University Sports Centre,La Trobe College Australia,"Level 3, Sylvia Walton Building, Bundoora VIC ...",Swimming Club,None,145.050119,-37.721764,"['university', 'library', 'point_of_interest',...",ChIJ6QUDCstC1moRtcFHQWj5wDs
3,Broadmeadows Swimming Clubs,NaN,Glenroy Bowls Club,"6 Ash Ct, Glenroy VIC 3046, Australia",Swimming Club,None,144.923334,-37.697146,"['point_of_interest', 'establishment']",ChIJEVPpFfpa1moRhRK6utD_-JI
4,Broadmeadows Swimming Clubs,NaN,Glenroy Football Club,"Sewell Reserve - Glenroy Road,, Glenroy VIC 30...",Swimming Club,None,144.933768,-37.707209,"['point_of_interest', 'establishment']",ChIJ0aW-aOlLJgARatzdoB2dnCc


In [50]:
sports_con = pd.concat([sports_c, sports_c2], axis=0, ignore_index=True)

In [51]:
sports_con.shape

(664, 7)

In [52]:
sports_con[sports_con.isnull().any(1)]

,Name,formattedAddress,newB,longitude,latitude,type,place_id


In [53]:
sports_con.head()

,Name,formattedAddress,newB,longitude,latitude,type,place_id
0,Eltham Swimming School,"1441 Main Rd, Eltham VIC 3095, Australia",Swimming Club,145.175774,-37.701532,"['point_of_interest', 'establishment']",ChIJnc-kCFw21moR-O84vUwjjTM
1,Jayson Lamb’s Splash Factory,"Grafton Rd, Warrnambool VIC 3280, Australia",Swimming Club,142.501409,-38.379052,"['point_of_interest', 'establishment']",ChIJV5zTXLpNzWoRKdMXeO49xpE
2,Just Swimming Wesley,"Wesley College, 554 High St Rd, Mount Waverley...",Swimming Club,145.146906,-37.872370,"['health', 'point_of_interest', 'school', 'est...",ChIJZ28toY0_1moRIKoSaJU36AM
3,Melbourne Vicentre Swimming Club,"Aughtie Dr, Albert Park VIC 3206, Australia",Swimming Club,144.960939,-37.841647,"['point_of_interest', 'establishment']",ChIJud2iLApo1moRpnfMpGO_mcQ
4,Melbourne Vicentre Swimming Club,"Ranfurlie Cres, Glen Iris VIC 3146, Australia",Swimming Club,145.054289,-37.861586,"['health', 'point_of_interest', 'school', 'est...",ChIJk4SJNWBA1moRxEt3tTgM20A


In [54]:
# write to csv for furthre processing
sports_con.to_csv('sports_concate_v1_0.csv', encoding='utf8')

In [ ]:
################ END ########################

In [ ]:
ss = set()
ss.add('string')
ss.add('list') 
ss.add('come')
ss

In [ ]:
ss = list(ss)

In [ ]:
ss

In [ ]:
tt = ','.join(e for e in ss)
tt

In [ ]:
# write
sports_t3.to_csv('extra.csv', encoding='utf8')

In [ ]:
sports1[sports1.iloc[:,:].type.str.contains('travel_agency')]

In [ ]:
count = 0
for i in range(len(swim.orgName.unique())):
    count += 1
count

In [ ]:
# check if there are overlapping orgName
swim_uni = swim.orgName.unique()
athl_uni = athlete.orgName.unique()
mart_uni = martial.orgName.unique()
park_uni = park.orgName.unique()

In [ ]:
for i in range(len(swim_uni)):
    for j in range(len(athl_uni)):
        if swim_uni[i] == athl_uni[j]:
            print(swim_uni[i])

In [ ]:
for i in range(len(swim_uni)):
    for j in range(len(mart_uni)):
        if swim_uni[i] == mart_uni[j]:
            print(swim_uni[i])

In [ ]:
for i in range(len(athl_uni)):
    for j in range(len(mart_uni)):
        if mart_uni[j] == athl_uni[i]:
            print(mart_uni[j])

In [ ]:
# check if there are overlapping Name in swim and athlete
swim_uni = swim.Name.unique()
athl_uni = athlete.Name.unique()
mart_uni = martial.Name.unique()
park_uni = park.Name.unique()

In [ ]:
for i in range(len(swim_uni)):
    for j in range(len(athl_uni)):
        if swim_uni[i] == athl_uni[j]:
            print(swim_uni[i])

In [ ]:
for i in range(len(swim_uni)):
    for j in range(len(mart_uni)):
        if swim_uni[i] == mart_uni[j]:
            print(swim_uni[i])

In [ ]:
for i in range(len(athl_uni)):
    for j in range(len(mart_uni)):
        if mart_uni[j] == athl_uni[i]:
            print(mart_uni[j])

In [ ]:
for i in range(len(athl_uni)):
    for j in range(len(park_uni)):
        if park_uni[j] == athl_uni[i]:
            print(park_uni[j])

In [ ]:
# check these two records
# Diamond Valley Little Athletics Centre
#swim[swim.Name == 'Diamond Valley Little Athletics Centre']

In [ ]:
#athlete [athlete.Name == 'Diamond Valley Little Athletics Centre']

In [ ]:
#swim[swim.Name == 'Oakleigh Recreation Centre']

In [ ]:
#athlete [athlete.Name == 'Oakleigh Recreation Centre']

In [ ]:
sports1[sports1.Name == 'Oakleigh Recreation Centre']

In [ ]:
sports1[sports1.Name == 'Diamond Valley Little Athletics Centre']

In [ ]:
# check type columns
sports1.type.unique()

In [ ]:
# check type: general_contractor
sports2 = sports1[sports1.iloc[:,:].type.str.contains('general_contractor')]
sports2
# we may not need this type

In [ ]:
# check type: university
sports2 = sports1[sports1.iloc[:,:].type.str.contains('university')]
sports2
# we may not need this type

In [ ]:
# check type: bar
sports2 = sports1[sports1.iloc[:,:].type.str.contains('travel_agency')]
sports2
# don't want this type

In [ ]:
# check type: food
sports2 = sports1[sports1.iloc[:,:].type.str.contains('food')]
sports2
# don't want this type

In [ ]:
sports1

In [ ]:
sports1.loc[929].formattedAddress

In [ ]:
# see how many unique locations in the data set
count = 0
for act  in sports1.place_id.unique():
    count += 1
count

In [ ]:
# check unique based on place_id
sports4 = sports1[sports1.duplicated(subset='place_id', keep=False)]
sports4

In [ ]:
#
for i in range(len(sports4.Name.unique())):
    

In [ ]:
count = 0
for i in range(sports4.shape[0]):
    count += 1
count

In [ ]:
sports1

In [ ]:
# type we do not need
nottype = ['bar','restaurant','food','store']


In [ ]:
sports2 = sports1[sports1.orgAddress.isnull()]
sports2

In [ ]:
# what is food type
sports5 = sports1[sports1.iloc[:,:].type.str.contains('food')]
sports5

In [ ]:
# fuzzy match with orgAddress and formateedAddress
# fuzzy match with orgName and Name




In [ ]:
st = 'a'
st2 = 'b'

In [ ]:
if st == 'a':
    break
elif st2 == 'b':
    print(b)

In [ ]:
# deal with the formatted address
# address format:
# Upper Ferntree Gully VIC 3156, Australia
# capture suburb and short name of state regex
#r'(([A-Z]{1}[a-zA-Z\s]+)[\s](VIC)[\s]([\d]{4}))'

# test
m = r'([A-Z]{1}[a-zA-Z\s]+)[\s](VIC)[\s]([\d]{4})'
add1 = 'Upper Ferntree Gully VIC 3156, Australia'
add2 = '149 Appin St, Wangaratta VIC 3677, Australia'
add3 = 'Riviera Reserve, Eel Race Rd, Seaford VIC 3198, Australia'


In [ ]:
f1 = re.findall(m, add1)
f2 = re.findall(m, add2)
f3 = re.findall(m, add3)
print(f1, f2, f3)
print(type(f1))


In [ ]:
findsub = r'([A-Z]{1}[a-zA-Z\s]+[a-z])[\s]VIC[\s][\d]{4}'
findpost = r'[A-Z]{1}[a-zA-Z\s]+[\s]VIC[\s]([\d]{4})'
f11 = re.findall(findsub, add1)
f12 = re.findall(findpost, add1)
f21 = re.findall(findsub, add2)
f22 = re.findall(findpost, add2)
f31 = re.findall(findsub, add3)
f32 = re.findall(findpost, add3)
print(f11, f12, f21, f22, f31, f32)
print(type(f1))

In [ ]:
f1[0]

In [ ]:
f11

In [ ]:
# The new list contains more club and facility types than the original one
# Sports Centre: generally have more than one activity
# Soccer Club
# Football Club
# Netball Club
# Tennis Club
# Swimming Club
# Martial Art: Judo, Taikwangdoe, , Taichi (usually for elderly, will not be considered)

# Park
# Swimming Pool
# 